### Quick Links
- [Mainnet Performance Example](#mainnet-example)
- [Testnet Performance Example](#testnet-example)

In [1]:
from PIL import Image
import matplotlib.pyplot as plt

from performance import compute_miner_performance
from helpers import formatted_ts_from_epoch, epoch_from_formatted_ts
from data import get_wanbd_runs

### Mainnet Example

In [2]:
project = "bitmind-subnet"  # mainnet wandb project name
entity = "bitmindai"
validator_run_name = 'validator-6-2.0.3'
start_dt = '2025-01-04T16:09:49'
# start_dt = formatted_ts_from_epoch(1729494162)
end_dt = None

miner_uid = 0
print(f'Querying data for uid {miner_uid} starting at {start_dt}')

Querying data for uid 0 starting at 2025-01-04T16:09:49


In [3]:
runs = get_wanbd_runs(entity, project, start_dt, end_dt, validator_run_name)
print(f'Fetched {len(runs)} runs')

Querying w&b with filters: {'display_name': 'validator-6-2.0.3', 'created_at': {'$gte': '2025-01-04T16:09:49'}}
Fetched 7 runs


In [4]:
results = compute_miner_performance(
    runs,
    miner_uid=miner_uid,  # example uid
    start_ts=epoch_from_formatted_ts(start_dt),
    end_ts=epoch_from_formatted_ts(end_dt),
    validator_run_name=validator_run_name)

results.keys()

dict_keys(['predictions', 'performance'])

In [7]:
preds_df = results['predictions']

# separate image/video preds if you wish
image_preds_df = preds_df[preds_df['modality'] == 'image']
video_preds_df = preds_df[preds_df['modality'] == 'video']

preds_df


,modality,uid,prediction,label,wandb_filepath,local_filepath
0,image,0,1,1,media/images/image_1_409a85d349a1338aa27d.png,not downloaded
1,video,0,0,0,media/videos/video_20_97c0e814f13b877483c0.gif,not downloaded
2,image,0,0,0,media/images/image_29_b82febbfa436c27ffbe8.png,not downloaded
3,video,0,1,1,media/videos/video_31_5f5af1a43efc63aad39d.gif,not downloaded
4,video,0,0,0,media/videos/video_33_afe4e212376fe3f1f62e.gif,not downloaded
...,...,...,...,...,...,...
199,video,0,0,0,media/videos/video_75_918c5e36aa003f9c2695.gif,not downloaded
200,video,0,0,0,media/videos/video_84_7b875e970294a7580801.gif,not downloaded
201,video,0,0,0,media/videos/video_101_c58de27e5003daa08db6.gif,not downloaded
202,video,0,0,0,media/videos/video_102_b17567b120135f474c0a.gif,not downloaded


In [6]:
results['performance']

,accuracy,precision,recall,f1,auc,sample_size,uid,modality
0,0.960784,0.937500,0.978261,0.957447,0.962345,102,0,image
1,0.960784,0.958333,0.958333,0.958333,0.960648,102,0,video


### Testnet Example

In [6]:
project = "bitmind"  # mainnet wandb project name
entity = "bitmindai"
validator_run_name = 'validator-39-2.0.3'
start_dt = '2025-01-04T16:09:49'
# start_dt = formatted_ts_from_epoch(1729494162)
end_dt = None

miner_uid = None
print(f'Querying data for uid {miner_uid} starting at {start_dt}')

Querying data for uid None starting at 2025-01-04T16:09:49


In [7]:
runs = get_wanbd_runs(entity, project, start_dt, end_dt, validator_run_name)
print(f'Fetched {len(runs)} runs')

Querying w&b with filters: {'display_name': 'validator-39-2.0.3', 'created_at': {'$gte': '2025-01-04T16:09:49'}}
Fetched 7 runs


In [8]:
results = compute_miner_performance(
    runs,
    miner_uid=miner_uid,  # example uid
    start_ts=epoch_from_formatted_ts(start_dt),
    end_ts=epoch_from_formatted_ts(end_dt),
    validator_run_name=validator_run_name)

results.keys()

dict_keys(['predictions', 'performance'])

In [9]:
preds_df = results['predictions']

# separate image/video preds if you wish
image_preds_df = preds_df[preds_df['modality'] == 'image']
video_preds_df = preds_df[preds_df['modality'] == 'video']

preds_df


,modality,uid,prediction,label,wandb_filepath,local_filepath
0,image,241,1.000000,1,media/images/image_1_84fd5414c6651754e6ec.png,not downloaded
1,image,241,0.000000,0,media/images/image_8_a744ac90c9df2dbb9d7f.png,not downloaded
2,image,241,0.000000,1,media/images/image_13_98a593b3ec189e7811cd.png,not downloaded
3,image,241,0.000000,0,media/images/image_16_cb8eb8b067ee99026937.png,not downloaded
4,image,241,1.000000,1,media/images/image_24_84fd5414c6651754e6ec.png,not downloaded
...,...,...,...,...,...,...
307,image,241,0.000000,0,media/images/image_124_e7d7eb1ca70fe0101106.png,not downloaded
308,image,248,0.010513,1,media/images/image_125_0eca6bbe8d65f27025fd.png,not downloaded
309,image,248,0.000122,1,media/images/image_128_a1fa18b585e93412b876.png,not downloaded
310,image,248,0.017804,0,media/images/image_132_0b4ae266c31d6176071e.png,not downloaded


In [10]:
results['performance']

,accuracy,precision,recall,f1,auc,sample_size,uid,modality
0,0.612903,1.000000,0.200000,0.333333,0.600000,155,241,image
1,0.631944,0.739130,0.265625,0.390805,0.700391,144,248,image
2,0.307692,0.307692,1.000000,0.470588,0.250000,13,248,video
